In [28]:
class_to_ix = {}
ix_to_class = {}
with open('food101/meta/classes.txt', 'r') as txt:
    classes = [l.strip() for l in txt.readlines()]
    class_to_ix = dict(zip(classes, range(len(classes))))
    ix_to_class = dict(zip(range(len(classes)), classes))
    class_to_ix = {v: k for k, v in ix_to_class.items()}
keys = []
for key,value in class_to_ix.items():
    keys.append(key)
print(np.random.choice(keys, size=10,replace=True))

['sushi' 'greek_salad' 'mussels' 'paella' 'omelette' 'frozen_yogurt'
 'beef_tartare' 'caesar_salad' 'bibimbap' 'filet_mignon']


In [7]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pretrainedmodels
import pretrainedmodels.utils
plt.ion() 

In [8]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=200,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

use_gpu = torch.cuda.is_available()

In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=50):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # Set model to training mode
            else:
                model.train(False)  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for data in dataloaders[phase]:
                # get the inputs
                inputs, labels = data

                # wrap them in Variable
                if use_gpu:
                    inputs = Variable(inputs.cuda())
                    labels = Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.data[0] * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best test Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model = pretrainedmodels.__dict__["inceptionresnetv2"](num_classes=1000,pretrained="imagenet")
features_size = model.last_linear.in_features
model.last_linear = nn.Linear(features_size,10)
if use_gpu:
    model = model.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=50)

Epoch 0/49
----------


In [ ]:
model_ft = models.resnet18()
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10)

if use_gpu:
    model_ft = model_ft.cuda()

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=50)

Epoch 0/49
----------
train Loss: 2.2415 Acc: 0.2193
test Loss: 1.9244 Acc: 0.3648

Epoch 1/49
----------
train Loss: 2.0477 Acc: 0.2951
test Loss: 1.8603 Acc: 0.3728

Epoch 2/49
----------
train Loss: 1.9645 Acc: 0.3340
test Loss: 1.9124 Acc: 0.4176

Epoch 3/49
----------
train Loss: 1.8766 Acc: 0.3545
test Loss: 1.5127 Acc: 0.5236

Epoch 4/49
----------
train Loss: 1.8365 Acc: 0.3835
test Loss: 1.5558 Acc: 0.5068

Epoch 5/49
----------
train Loss: 1.7715 Acc: 0.4019
test Loss: 1.4355 Acc: 0.5360

Epoch 6/49
----------
train Loss: 1.6953 Acc: 0.4333
test Loss: 1.3561 Acc: 0.5600

Epoch 7/49
----------
train Loss: 1.4775 Acc: 0.5109
test Loss: 1.0911 Acc: 0.6460

Epoch 8/49
----------
train Loss: 1.4628 Acc: 0.5195
test Loss: 1.0903 Acc: 0.6420

Epoch 9/49
----------
train Loss: 1.4151 Acc: 0.5352
test Loss: 1.0412 Acc: 0.6560

Epoch 10/49
----------
train Loss: 1.4110 Acc: 0.5356
test Loss: 1.0556 Acc: 0.6520

Epoch 11/49
----------
train Loss: 1.3884 Acc: 0.5483
test Loss: 1.0362 Acc

In [9]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=34)

Epoch 0/33
----------
train Loss: 1.3005 Acc: 0.5697
test Loss: 0.9390 Acc: 0.6980

Epoch 1/33
----------
train Loss: 1.3268 Acc: 0.5675
test Loss: 0.9477 Acc: 0.6936

Epoch 2/33
----------
train Loss: 1.3024 Acc: 0.5705
test Loss: 0.9782 Acc: 0.6836

Epoch 3/33
----------
train Loss: 1.2996 Acc: 0.5724
test Loss: 0.9526 Acc: 0.6928

Epoch 4/33
----------
train Loss: 1.2986 Acc: 0.5780
test Loss: 0.9413 Acc: 0.6912

Epoch 5/33
----------
train Loss: 1.3115 Acc: 0.5735
test Loss: 0.9517 Acc: 0.6896

Epoch 6/33
----------
train Loss: 1.3010 Acc: 0.5735
test Loss: 0.9634 Acc: 0.6920

Epoch 7/33
----------
train Loss: 1.3013 Acc: 0.5727
test Loss: 0.9578 Acc: 0.6864

Epoch 8/33
----------
train Loss: 1.3169 Acc: 0.5719
test Loss: 0.9501 Acc: 0.6872

Epoch 9/33
----------
train Loss: 1.3112 Acc: 0.5721
test Loss: 0.9480 Acc: 0.6988

Epoch 10/33
----------
train Loss: 1.2919 Acc: 0.5839
test Loss: 0.9779 Acc: 0.6920

Epoch 11/33
----------
train Loss: 1.3135 Acc: 0.5737
test Loss: 0.9360 Acc